In [11]:
import pyspark
from pyspark.sql import SparkSession
import os

os.environ['AWS_REGION'] = "eu-west-1"
conf = (
    pyspark.SparkConf()
        .setAppName('iceberg-data-explorer')
        .set("spark.sql.catalog.rest", "org.apache.iceberg.spark.SparkCatalog")
        .set("spark.sql.catalog.rest.catalog-impl", "org.apache.iceberg.rest.RESTCatalog")
        .set("spark.sql.catalog.rest.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .set("spark.sql.catalog.rest.uri", "http://iceberg-catalog:8181/")
        .set("spark.sql.catalog.rest.s3.endpoint", "http://minio:9000")
        .set("spark.sql.catalog.rest.s3.path-style-access", "true")
        .set("spark.sql.defaultCatalog", "rest")
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

Spark Running


In [12]:
spark.sql("SELECT * FROM iceberg.application_db_public_products").show()

+---+--------------+------------+----+--------+--------------+-----------+---------+--------------------+
| id|          name|       price|__op| __table|__source_ts_ms|       __db|__deleted|         __source_ts|
+---+--------------+------------+----+--------+--------------+-----------+---------+--------------------+
|  5|         Shoes|{2, [00 8F]}|   u|products| 1695380439311|application|    false|2023-09-22 11:00:...|
|  1|          Book|{2, [02 6E]}|   d|products| 1695380454902|application|     true|2023-09-22 11:00:...|
|  3|Remote Control|{2, [04 13]}|   d|products| 1695295333326|application|     true|2023-09-21 11:22:...|
|  2|         Chair|{2, [09 CD]}|   r|products| 1695288693878|application|    false|2023-09-21 09:31:...|
|  3|Remote Control|{2, [04 13]}|   c|products| 1695380350829|application|    false|2023-09-22 10:59:...|
+---+--------------+------------+----+--------+--------------+-----------+---------+--------------------+



In [8]:
spark.sql("""
CREATE TABLE iceberg.order (
    id bigint COMMENT 'unique id',
    timestamp timestamp,
    product_id bigint,
    quantity bigint)
USING iceberg
PARTITIONED BY (bucket(16, id), days(timestamp))
"""
)

DataFrame[]

In [5]:
spark.sql("""
DROP TABLE iceberg.order PURGE
""")

DataFrame[]

In [17]:
spark.sql("""
INSERT INTO iceberg.order VALUES 
(1, now(), 5, 10), 
(2, now(), 3, 24)
""")

DataFrame[]

In [18]:
spark.sql("SELECT * FROM iceberg.order").show()

+---+--------------------+----------+--------+
| id|           timestamp|product_id|quantity|
+---+--------------------+----------+--------+
|  1|2023-09-22 11:54:...|         5|      10|
|  2|2023-09-22 11:54:...|         3|      24|
+---+--------------------+----------+--------+



In [15]:
spark.sql("""
DELETE FROM iceberg.order
WHERE id IN ('1', '2')
""")

DataFrame[]

In [23]:
spark.sql("""
SELECT * FROM iceberg.order.history;
""").show()

spark.sql("""
SELECT * from iceberg.order.metadata_log_entries;
""").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2023-09-22 11:52:...|7887498953345238229|               NULL|               true|
|2023-09-22 11:52:...|1663266378830185026|7887498953345238229|               true|
|2023-09-22 11:54:...|8445021180767544983|1663266378830185026|               true|
|2023-09-22 11:54:...|3781607614289642584|8445021180767544983|               true|
+--------------------+-------------------+-------------------+-------------------+

+--------------------+--------------------+-------------------+----------------+----------------------+
|           timestamp|                file| latest_snapshot_id|latest_schema_id|latest_sequence_number|
+--------------------+--------------------+-------------------+----------------+----------------------+
|2023-09-22 11:50:...|s

In [25]:
spark.sql("""
SELECT * from iceberg.order o
JOIN iceberg.application_db_public_products p
  on o.product_id = p.id
WHERE p.__deleted = false
""").show()

+---+--------------------+----------+--------+---+--------------+------------+----+--------+--------------+-----------+---------+--------------------+
| id|           timestamp|product_id|quantity| id|          name|       price|__op| __table|__source_ts_ms|       __db|__deleted|         __source_ts|
+---+--------------------+----------+--------+---+--------------+------------+----+--------+--------------+-----------+---------+--------------------+
|  1|2023-09-22 11:54:...|         5|      10|  5|         Shoes|{2, [00 8F]}|   u|products| 1695380439311|application|    false|2023-09-22 11:00:...|
|  2|2023-09-22 11:54:...|         3|      24|  3|Remote Control|{2, [04 13]}|   c|products| 1695380350829|application|    false|2023-09-22 10:59:...|
+---+--------------------+----------+--------+---+--------------+------------+----+--------+--------------+-----------+---------+--------------------+

